In [7]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

from packages.functions import set_env_variables, instantiate_llm

set_env_variables()

It's all right!


In [8]:
llm = instantiate_llm(model_name="gpt-3.5-turbo")

In [9]:
with open('../section-02/files/winston_churcill_speech.txt', encoding='utf-8') as f:
    text = f.read()

llm.get_num_tokens(text)

4910

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=12000, chunk_overlap=50)
chunks = text_splitter.create_documents([text])

len(chunks)


2

In [11]:
chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    verbose=False
)

output = chain.run(chunks)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-va3LEBvnLYt8QWkUrWq6m1TC on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


In [12]:
print(output)

This passage describes the successful evacuation of British and French troops from the beaches of France and Belgium during World War II, with the assistance of the Royal Air Force and Navy. The author emphasizes the need for defense against potential invasion and the determination to continue the fight against tyranny. The passage also mentions the losses suffered by the British Army and the measures being taken to strengthen defenses.


This passage describes the successful evacuation of British and French troops from the beaches of France and Belgium during World War II, with the assistance of the Royal Air Force and Navy. The author emphasizes the need for defense against potential invasion and the determination to continue the fight against tyranny. The passage also mentions the losses suffered by the British Army and the measures being taken to strengthen defenses.

## map_reduce with Custom Prompts

In [18]:
map_prompt = """
    Write a short and concise summary of the following:
    Text: `{text}`
    CONCISE SUMMARY:
"""

map_prompt_template = PromptTemplate(
    input_variables=["text"],
    template=map_prompt
)

In [19]:
combine_prompt = """
    Write a concise summary of the following text that covers the key points.
    Add a title to the summary.
    Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
    Text: `{text}`
"""

combine_prompt_template = PromptTemplate(
    input_variables=["text"],
    template=combine_prompt
)

In [20]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=False
)

output = summary_chain.run(chunks)

In [21]:
print(output)

Title: The Evacuation of Dunkirk and Churchill's Speech

Introduction:
This text discusses the evacuation of British and Allied troops from Dunkirk during World War II, known as Operation Dynamo. It highlights the challenges faced by the British forces and the crucial role played by the Royal Navy and Royal Air Force in the successful evacuation.

Key Points:
- Operation Dynamo began on May 26, evacuating over 338,000 Allied troops, including 26,000 French soldiers, to England.
- The British Royal Air Force fought against heavy attacks to deny the enemy total air supremacy.
- The surrender of the Belgian Army and the heroic defense of Boulogne and Calais are mentioned.
- The Royal Navy and merchant seamen played a crucial role in the evacuation.
- The evacuation showcased the achievements of the British Air Force, despite not being a victory.

Conclusion:
Winston Churchill's speech acknowledges the challenges faced by the British Army and emphasizes the need to strengthen defenses and 